# Working with different authentication schemes

The `GIS` class in the ArcGIS API for Python supports several classes of users with varied authentication schemes:
* [Anonymous users](#Anonymous-users)
* [Built-in users](#Built-in-users)
* [Enterprise identity store](#Enterprise-identity-store)
  * [Web-tier authentication with LDAP](#Web-tier-authentication-with-LDAP)
  * [Portal-tier authentication with Active Directory](#Portal-tier-authentication-with-Active-Directory)
  * [Portal-tier authentication with LDAP](#Portal-tier-authentication-with-LDAP)
  * [Web-tier authentication secured with PKI](#Web-tier-authentication-secured-with-PKI)
  * [Web-tier authentication secured with IWA](#Web-tier-authentication-secured-with-IWA)
* [User authentication with OAuth 2.0](#User-authentication-with-OAuth-2.0)
* [Connecting through ArcGIS Pro](#Connecting-through-ArcGIS-Pro)
* [Protecting your credentials](#Protecting-your-credentials)
* [Storing your credentials locally](#Storing-your-credentials-locally)

The `GIS` class supports constructing a `GIS` object given an url and user credentials to ArcGIS Online or an ArcGIS Enterprise instance. User credentials can be passed in using username / password pair, or using a key_file / cert_file pair (in case of PKI). The `GIS` class also supports built-in users, LDAP, PKI and anonymous access.

If no url is provided, ArcGIS Online is used. If username / password or key / cert files are not provided, anonymous access is used. Additionally, the script can leverage the ArcGIS Pro app to login to the portal and their scripts can use whichever Portal is currently active.

The examples below demonstrate how to connect using these different authentication schemes:

In [1]:
# import the GIS class in gis module
from arcgis.gis import GIS

## Anonymous users
You can make use of the Python API as anonymous user. However, tasks that you can accomplish are fewer, as you would not be able to create or modify content or perform any analysis tasks. As an anonymous user, you can query and view publicly shared maps and layers.
### Connect to ArcGIS Online as an anonymous user

In [2]:
print("ArcGIS Online as anonymous user")    
gis = GIS()
print("Logged in as anonymous user to " + gis.properties.portalName)

ArcGIS Online as anonymous user
Logged in as anonymous user to ArcGIS Online


### Connect to your ArcGIS Enterprise as an anonymous user
If your ArcGIS Enterprise allows anonymous usage, you can connect to it as such by specifying the URL and leaving the username and password parameters empty. 

<blockquote><b>Note</b>: If your enterprise is configured with IWA authentication, and if you are running this on a Windows computer, you might get automatically signed in.</blockquote>

In [3]:
gis = GIS("https://python.playground.esri.com/portal")
print("Logged in as anonymous user to " + gis.properties.portalName)

Logged in as anonymous user to Portal for ArcGIS


## Built-in users
Both ArcGIS Online and ArcGIS Enterprise come preconfigured with a [built-in identity store](http://server.arcgis.com/en/portal/latest/administer/windows/about-configuring-portal-authentication.htm#ESRI_SECTION1_83F7B85FEF594A6B96997AF3CADF3D38), so you can easily create accounts and groups in your portal. When signing in to such accounts, specify your username and password, as you type them into the portal website.

### Connect to ArcGIS Online using a built-in account

In [4]:
print("ArcGIS Online Org account")    
gis = GIS("https://www.arcgis.com", "arcgis_python", "P@ssword123")
print("Logged in as " + str(gis.properties.user.username))

ArcGIS Online Org account
Logged in as arcgis_python


### Connect to ArcGIS Enterprise using a built-in account

In [ ]:
print("Portal for ArcGIS as a built in user")
gis = GIS("https://portalname.domain.com/webadapter_name", "sharinguser", "password")
print("Logged in as: " + gis.properties.user.username)

Portal for ArcGIS as a built in users
Logged in as sharinguser


## Enterprise identity store
ArcGIS Online and Enterprise is designed so you can use your enterprise accounts and groups to control access to your ArcGIS organization.
### Web-tier authentication with LDAP

In [ ]:
print("\n\nBasic Authentication with LDAP")    
ldapbasic = GIS("https://portalname.domain.com/webadapter_name", "amy", "password")
print("Logged in as: " + ldapbasic.properties.user.username)



Basic Authentication with LDAP
Logged in as: amy


### Portal-tier authentication with Active Directory
When connecting using an enterprise user account from an Active Directory, specify your username as `domain\\username`

In [ ]:
print("\n\nPortal-tier Authentication with LDAP - enterprise user")
gisldap = GIS("https://portalname.domain.com/webadapter_name", "AVWORLD\\Publisher", "password")
print("Logged in as: " + gisldap.properties.user.username)



Portal-tier Authentication with LDAP - enterprise user
Logged in as: Publisher@AVWORLD


### Portal-tier authentication with LDAP
When connecting using an enterprise user account from LDAP, specify your username and password as usual

In [ ]:
print("\n\nPortal-tier Authentication with LDAP - builtin user")    
gisldap = GIS("https://portalname.domain.com/webadapter_name", "sharing1", "password")
print("Logged in as: " + gisldap.properties.user.username)



Portal-tier Authentication with LDAP - builtin user
Logged in as: sharing1


### Web-tier authentication secured with PKI

You can also specify a local certificate and a 'key' file to use as client side certificate, when logging into your ArcGIS Enterprise using PKI-based client certificate authentication.

In [ ]:
print("\n\nPKI with key and cert files")  
gis = GIS("https://portalname.domain.com/webcontext", 
          key_file="C:\\path\\to\\key.pem",
          cert_file="C:\\path\\to\\cert.pem")
print("Logged in as: " + gis.properties.user.username)



PKI with key and cert files
Logged in as: username@DOMAIN


### Web-tier authentication secured with IWA

If your portal is configured to pick up your Windows credentials using NTLM or Kerberos, you can omit passing in the username and password. The ArcGIS API for Python is able to figure out when the GIS is using Windows authentication and picks the login credentials from the currently running process providing a seamless and secure login experience. 

<blockquote>
Windows authentication only works on the Windows OS and requires pywin32 and kerberos-sspi python packages.</blockquote>

In [ ]:
print("\n\nIntegrated Windows Authentication using NTLM or Kerberos")  
gis = GIS("https://portalname.domain.com/webcontext")
print("Logged in as: " + gis.properties.user.username)



Integrated Windows Authentication using NTLM or Kerberos
Logged in as: username@DOMAIN


## User authentication with OAuth 2.0 
The ArcGIS Python API supports [OAuth 2.0](https://developers.arcgis.com/documentation/core-concepts/security-and-authentication/what-is-oauth-2/) as an authentication method, and acts as a [serverless native application](https://developers.arcgis.com/documentation/core-concepts/security-and-authentication/mobile-and-native-user-logins/) when using OAuth 2.0 authorization with ArcGIS.

To use this mode of authorization, you need a `client id`. If you already have a client id, you can skip the following section.

### Obtaining a client id
The steps below show how a client id can be obtained by registering a new application with your GIS. Only one client id is required, so if your GIS has one already, you may use it instead of creating a new application.
    
* Log into your web GIS (ArcGIS Online organization or ArcGIS Enterprise)
* Go to Content tab
* Click '+ Add Item > An Application' menu
* Add an application:
 * Type: Application
 * Title: Python
 * Tags: python
* On the Item details page of this newly created application, navigate to Settings tab
* Click the Registered Info button. It's towards the bottom of the page.
* This will dispaly an App ID. This is your client id and needs to be passed in as the client_id parameter when construcing a GIS object. You need this in your Python code to log in.
    
You can then log on to your org using the Python API using the code shown below:

This uses interactive sign-in experience: you would be redirected to your organization's sign in page using the configured identity provider. Upon signing in, you would get a code, that you can paste back to complete the sign in process:

In [9]:
gis = GIS("https://python.playground.esri.com/portal", client_id='f8cRxbP3NO8bf9ag')
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://python.playground.esri.com/portal/sharing/rest/oauth2/authorize?client_id=f8cRxbP3NO8bf9ag&response_type=code&expiration=-1&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob
Enter code obtained on signing in using SAML: ········
Successfully logged in as: arcgis_python


If the configured identity provider is compatible, you may also be able to provide your username and password, along with the client_id for a non interactive login experience using OAuth 2.0, as shown below:

In [10]:
gis = GIS("https://python.playground.esri.com/portal", username="arcgis_python", password="amazing_arcgis_123",
          client_id='f8cRxbP3NO8bf9ag')
print("Successfully logged in as: " + gis.properties.user.username)

Successfully logged in as: arcgis_python


## Connecting through ArcGIS Pro

It is often useful to write scripts that work against the active portal in the ArcGIS Pro app.

Using the `pro` authentication scheme, scripts can create an instance of the `GIS` class representing the active portal in ArcGIS Pro without requiring the user to pass their credentials. In this mode, users can leverage the Pro app to login to the portal and their scripts can use whichever Portal is currently active. This mode can also serve as a bridge for users with advanced authentication scenarios like IWA using NTLM or Kerberos or Smart Card where signing in with credentials may not be possible or desirable.

Please note that ArcGIS Pro should be installed and concurrently running when the script is executed, for this mode to work.

In [ ]:
print("\n\nActive Portal in ArcGIS Pro")  
gis = GIS("pro")



Active Portal in ArcGIS Pro


## Protecting your credentials

When sharing your notebooks with other users or if you are storing it in a public location you want to obfuscate your credentials stored in the notebook. You can do so by simply not providing the password. The Python API uses the `getpass` module internally and would request your password in such a way that your keystrokes would not be printed on the screen.

In [13]:
gis = GIS("https://python.playground.esri.com/portal", username='arcgis_python')
print("Successfully logged in as: " + gis.properties.user.username)

Enter password: ········
Successfully logged in as: arcgis_python


This halts execution of the cell and waits for the user to input text into  text box. All keystrokes are masked with an asterisk or similar character. Upon entry, execution resumes. The value entered by the user can be stored in a variable as shown in the code above.

This technique could be used with any authentication scheme that requires a password. You can choose to mask the username as well. The `getpass()` function can be used in a standalone Python script as well. When doing so, remember, the script has to be run interactively as it expects input from user at runtime.

## Storing your credentialls locally
If you are signing in frequently to a particular GIS and would like to store the credentials locally on your computer, then you could do so using the `profile` parameter of your `GIS`.

<blockquote>
This persisted profile for the GIS is stored in the users home directory in a config file named .arcgisprofile. The profile is <b>NOT ENCRYPTED</b> and you need to take care to protect the saved profile using operating system security or by other means. Once a profile has been saved, passing the profile parameter by itself uses the authorization credentials saved in the configuration file.
</blockquote>

In [23]:
gis = GIS("https://python.playground.esri.com/portal", username='arcgis_python', password='amazing_arcgis_123',
          profile='python_playground_prof')

print("Successfully logged in as: " + gis.properties.user.username)

Successfully logged in as: arcgis_python


To sign in during a different session, simply specify the profile file name.

In [24]:
gis2 = GIS(profile='python_playground_prof')
print("Successfully logged in as: " + gis2.properties.user.username)

Successfully logged in as: arcgis_python
